In [1]:
from bs4 import BeautifulSoup as bs
import requests
import random
import time
import re
import pandas as pd

In [2]:
def get_car_name(car):
    car_title = 'N/A'
    try:
        car_title = car.find('div', {'class', 'tmm-sf-search-card-list-view__title tmm-sf-search-card-list-view__title--bold'}).text
    except AttributeError:
        print('missing title')
    return car_title

In [3]:
def get_car_detail(car):
    car_km = 'N/A'
    liter_fuel = 'N/A'
    car_type = 'N/A'
    try:
        car_detail = car.find_all('div', {'class', 'tmm-sf-search-card-list-view__attributes'})
        if (len(car_detail) > 2):
            car_km = car_detail[0].text
            liter_fuel = car_detail[1].text
            car_type = car_detail[2].text
        else:
            car_km = car_detail[0].text
            liter_fuel = car_detail[1].text
            car_type = 'N/A'
        
    except (AttributeError, IndexError):
        print('missing detail')
    return {
            "km" : car_km,
            "liter and fuel" : liter_fuel,
            "type" : car_type
           }

In [4]:
def get_car_model(car):
    car_model = 'N/A'
    try:
        car_model = car.find('div', {'class', 'tmm-sf-search-card-list-view__text-model-detail tmm-sf-search-card-list-view__text-bold'}).text
    except AttributeError:
        print('missing model')
    return car_model

In [5]:
def get_car_description(car):
    car_description = 'N/A'
    try:
        car_description = car.find('div', {'class', 'tmm-sf-search-card-list-view__text-subtitle'}).text
    except AttributeError:
        print('missing description')
    return car_description

In [6]:
def get_car_location(car):
    car_location = 'N/A'
    try:
        car_location = car.find('span', {'class', 'tmm-sf-search-card-list-view__listing-date-location'}).text
    except AttributeError:
        print('missing location')
    return car_location

In [7]:
def get_car_seller(car):
    car_seller = 'N/A'
    try:
        car_seller = car.find('span', {'class', 'tmm-sf-search-card-list-view__profile-nickname'}).text
    except AttributeError:
        print('car dealer')
    return car_seller

In [8]:
def get_car_askingprice(car):
    car_askingprice = 'N/A'
    try:
        car_askingprice = car.find('span', {'class', 'tmm-sf-search-card-list-view__price-amount'}).text
    except AttributeError:
        print('missing price')
    return car_askingprice

In [9]:
def get_car_oldprice(car):
    car_oldprice = 'N/A'
    try:
        car_oldprice = car.find('span', {'class', 'tmm-sf-search-card-list-view__was-price'}).text
    except AttributeError:
        print('no discount')
    return car_oldprice

In [10]:
def get_car_pricelabel(car):
    on_road_cost = 'N/A'
    try:
        car_pricelabel = car.find_all('span', {'class', 'tmm-sf-search-card-list-view__price-label'})
        if (len(car_pricelabel) == 3):
            on_road_cost = car_pricelabel[2].text
        else:
            on_road_cost = car_pricelabel[1].text
    except (AttributeError, IndexError):
        print('missing label')       
    return on_road_cost

In [11]:
def scrap(url,link,header):
    car_dict = {}
    index = 1
    for page in range(1,71):
            
        wait_time = random.uniform(7,16)
        time.sleep(wait_time)
        print ("Scraping: " + url + link + '&page=' + str(page) + "&sort_order=mtr_best_match")
        response = requests.get(url + link + '&page=' + str(page) + "&sort_order=mtr_best_match", headers=header)
        html = response.content
        page_soup = bs(html, 'lxml')
        cars = page_soup.find_all('div', {'class' : 'tmm-sf-search-card-list-view__card'})
            
        if cars is None:
            break
            
        
        rank_count = 1
        for car in cars:
            car_info = get_car_detail(car)
            car_dict[index] = {
                    "name" : get_car_name(car).replace('\n', ' ').replace('\r', ''),
                    "km" : car_info["km"].replace('\n', ' ').replace('\r', ''),
                    "liter_fuel" : car_info["liter and fuel"].replace('\n', ' ').replace('\r', ''),
                    "type" : car_info["type"].replace('\n', ' ').replace('\r', ''),
                    "model" : get_car_model(car),
                    "description" : get_car_description(car),
                    "location" : get_car_location(car),
                    "seller" : get_car_seller(car),
                    "asking_price" : get_car_askingprice(car),
                    "old_price" : get_car_oldprice(car),
                    "on_road_cost" : get_car_pricelabel(car).replace('\n', ' ').replace('\r', '').replace('\t', ''),
                    "page" : page,
                    "page_rank" : rank_count
                    }
            rank_count = rank_count + 1
            index = index + 1
        print("Number of products scrapped: " + str(len(car_dict)))
        
    return car_dict

In [ ]:
url = "https://www.trademe.co.nz"
link = "/browse/categoryattributesearchresults.aspx?searchregion=100&cid=268&search=1&v=list&nofilters=1&originalsidebar=1&rptpath=1-268-&rsqid=8b2b6a0029a44666931cce5b729b4f9d-005&key=879991210"
header={'user-agent':''}

cars = scrap(url,link,header)


Scraping: https://www.trademe.co.nz/browse/categoryattributesearchresults.aspx?searchregion=100&cid=268&search=1&v=list&nofilters=1&originalsidebar=1&rptpath=1-268-&rsqid=8b2b6a0029a44666931cce5b729b4f9d-005&key=879991210&page=1&sort_order=mtr_best_match
missing description
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
no discount
missing description
no discount
no discount
no discount
no discount
missing description
no discount
no discount
missing description
no discount
missing model
no discount
missing model
no discount
missing model
no discount
missing model
no discount
car dealer
no discount
no discount
no discount
no discount
no discount
missing description
car dealer
missing description
no discount
no discount
missing description
car dealer
no discount
no discount
car dealer
missing model
missing description
no discount
no discount
missing description
no discount
no discount
missing model
no discount
car dealer
no discount
car dealer
no discoun

missing description
car dealer
missing description
car dealer
car dealer
car dealer
Number of products scrapped: 300
Scraping: https://www.trademe.co.nz/browse/categoryattributesearchresults.aspx?searchregion=100&cid=268&search=1&v=list&nofilters=1&originalsidebar=1&rptpath=1-268-&rsqid=8b2b6a0029a44666931cce5b729b4f9d-005&key=879991210&page=7&sort_order=mtr_best_match
missing model
no discount
car dealer
no discount
car dealer
no discount
missing description
car dealer
no discount
missing model
no discount
car dealer
no discount
missing description
no discount
no discount
missing model
no discount
missing model
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
no discount
no discount
car dealer
no discount
no discount
no discount
no discount
missing model
no discount
missing model
no discount
no discount
missing model
no discount
car dealer
no discount
missing model
no discount
no discount
no discount
missing model
no discount
missing model
no discount
m

missing model
no discount
missing description
car dealer
no discount
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
car dealer
car dealer
car dealer
no discount
missing description
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
Number of products scrapped: 600
Scraping: https://www.trademe.co.nz/browse/categoryattributesearchresults.aspx?searchregion=100&cid=268&search=1&v=list&nofilters=1&originalsidebar=1&rptpath=1-268-&rsqid=8b2b6a0029a44666931cce5b729b4f9d-005&key=879991210&page=13&sort_order=mtr_best_match
no discount
car dealer
car dealer
no discount
no discount
no discount
missing description
car dealer
no discount
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
missing description
car dealer
no discount
car dealer
no discount
car dealer
car dealer
car dealer
car dealer
no discount
car dealer
no d

car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
no discount
no discount
car dealer
no discount
car dealer
car dealer
no discount
car dealer
car dealer
no discount
missing description
car dealer
no discount
car dealer
no discount
missing description
car dealer
no discount
car dealer
car dealer
missing description
car dealer
no discount
missing model
no discount
no discount
missing description
car dealer
no discount
car dealer
no discount
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing model
car dealer
no discount
car dealer
no discount
Number of products scrapped: 900
Scraping: https://www.trademe.co.nz/browse/

Scraping: https://www.trademe.co.nz/browse/categoryattributesearchresults.aspx?searchregion=100&cid=268&search=1&v=list&nofilters=1&originalsidebar=1&rptpath=1-268-&rsqid=8b2b6a0029a44666931cce5b729b4f9d-005&key=879991210&page=24&sort_order=mtr_best_match
missing description
car dealer
no discount
car dealer
no discount
car dealer
no discount
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
no discount
missing description
car dealer
no discount
missing description
no discount
car dealer
no discount
missing description
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
no discount
car dealer
no discount
car dealer
no discount
no discount
car dealer
no discount
no discount
car dealer
no discount
missing model
no discount
missing model
missing description
no discount
no discount
no discount
car dealer
no discount
no discount
missing model
no discount
no discount
missing d

car dealer
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
Number of products scrapped: 1450
Scraping: https://www.trademe.co.nz/browse/categoryattributesearchresults.aspx?searchregion=100&cid=268&search=1&v=list&nofilters=1&originalsidebar=1&rptpath=1-268-&rsqid=8b2b6a0029a44666931cce5b729b4f9d-005&key=879991210&page=30&sort_order=mtr_best_match
car dealer
no discount
car dealer
no discount
car dealer
no discount
missing model
car dealer
no discount
missing model
missing description
no discount
missing description
no discount
no discount
no d

missing description
car dealer
no discount
missing description
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
missing description
car dealer
no discount
missing description
car dealer
no discount
car dealer
missing description
car dealer
no discount
missing description
car dealer
no discount
missing description
car dealer
no discount
car dealer
no discount
missing description
car dealer
no discount
car dealer
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
car dealer
no discount
missing description
car dealer
no discount
car dealer
no discount
car dealer
no discou

In [22]:
cars


{1: {'name': '                     2013 Mazda Axela                 ',
  'km': '                     137,459km                 ',
  'liter_fuel': '                     1997cc, Petrol                 ',
  'type': 'N/A',
  'model': 'N/A',
  'description': '2013 Mazda Axela',
  'location': ' North Shore - Auckland',
  'seller': 'claired',
  'asking_price': '$12,500 ',
  'old_price': 'N/A',
  'on_road_cost': '     Includes On Road Costs     ',
  'page': 1,
  'page_rank': 1},
 2: {'name': '                     2008 BMW X3                 ',
  'km': '                     121,901km                 ',
  'liter_fuel': '                     3000cc, Diesel                 ',
  'type': '                     Automatic                 ',
  'model': '3.0D NZ New Low Kms',
  'description': 'ON-SITE FINANCE AT NO DEPOSIT',
  'location': ' Christchurch City - Canterbury',
  'seller': 'N/A',
  'asking_price': '$14,990',
  'old_price': 'N/A',
  'on_road_cost': '     Includes On Road Costs     ',
  'page':

In [26]:
df = pd.DataFrame.from_dict(cars, orient='index')
df.to_csv('trademecars.csv')